### Calling an LLM

Sammo makes it easy to call an LLM and present the results in a structured way.

In [9]:
from sammo.runners import OpenAIChat
from sammo.components import GenerateText, Output
import os

runner = OpenAIChat(
    model_id="gpt-4o-mini",
    api_config={"api_key": os.environ['OPENAI_API_KEY']},
    cache=os.getenv("CACHE_FILE", "cache.tsv"),
    timeout=30,
)

first = GenerateText(
    "Hello! My name is Mike and I work as a prompt engineer.",
    system_prompt="Talk like Shakespeare.",
)
Output(first).run(runner)

+---------+------------------------------------------------------------+
| input   | output                                                     |
+=========+============================================================+
| None    | Hark! Good morrow, fair Mike! Thou art a prompt engineer,  |
|         | thou sayest? A noble craft indeed, for to weave words and  |
|         | summon forth the musings of the mind is a skill most rare. |
|         | Pray, what dost thou seek to conjure with thy art?         |
+---------+------------------------------------------------------------+
Constants: None

In [10]:
second = GenerateText("Write a four line poem about my job.", history=first)
poem = Output(second).run(runner)
poem

+---------+-------------------------------------------------------------+
| input   | output                                                      |
+=========+=============================================================+
| None    | In realms of thought where prompts take flight,   Thou dost |
|         | craft the words that spark the light.   With deftest hand,  |
|         | thou guidest minds anew,   A weaver of dreams, in all that  |
|         | thou dost do.                                               |
+---------+-------------------------------------------------------------+
Constants: None

In [11]:
poem.outputs[0].plot_call_trace()

### Static Loops

You can loop over a static list of inputs and then combine them with Union.

In [20]:
from sammo.components import Union

N = 5
fruits = [
    GenerateText("Generate the name of an LLM model provided by OpenAI. Only respond with the name of the model.", randomness=0.9, seed=i)
    for i in range(N)
]
static_loop = Output(Union(*fruits))
static_loop.run(runner)

+---------+-------------------------------------------------+
| input   | output                                          |
+=========+=================================================+
| None    | ['GPT-4', 'GPT-4', 'GPT-4', 'GPT-4', 'GPT-3.5'] |
+---------+-------------------------------------------------+
Constants: None

In [21]:
static_loop.plot_program()

### Dynamic Loops

You can loop over all the results of the previous layer with ForEach.

In [25]:
from sammo.extractors import ExtractRegex
from sammo.components import ForEach
from sammo.base import Template

models = ExtractRegex(
    GenerateText(
        "Generate a list of 5 models by OpenAI. Wrap each model with <model> and </model>."
    ),
    r"<model>(.*?)<.?model>"
)

model_blurbs = Output(ForEach(
    "model",
    models,
    GenerateText(Template("Why is {{model}} a good model in less than 25 words?")),
))
model_desc = model_blurbs.run(runner)

In [26]:
model_desc.outputs[0].plot_call_trace()


In [27]:
model_blurbs.plot_program()


### Parallelization

Sammo automatically runs components in parallel where possible within a rate limit.

In [35]:
from sammo.data import DataTable

runner = OpenAIChat(
    model_id="gpt-4o-mini",
    api_config={"api_key": os.environ["OPENAI_API_KEY"]},
    cache=os.getenv("CACHE_FILE", "cache.tsv"),
    rate_limit=6 # default is 2 per second
)
numbers = list(range(1,6))
spp = Output(GenerateText(Template("Output only the corresponding greek letter in English: {{input}}")))
spp.run(runner, DataTable(numbers))


minibatches[###################################################################################]5/5[00:00<00:00, 6.32it/s]


+---------+----------+
| input   | output   |
+=========+==========+
| 1       | Alpha    |
+---------+----------+
| 2       | Beta     |
+---------+----------+
| 3       | Gamma    |
+---------+----------+
| 4       | Delta    |
+---------+----------+
| 5       | Epsilon  |
+---------+----------+
Constants: None